In [1]:
import requests
import pandas as pd
import os
import re
from xbbg import blp

# proxies = {
# 'http':'http://  ',
# 'https':'http://   '
# }
# response = requests.get(url, proxies = proxies, timeout=60, verify=False)

path0 = r'C:\Users\OneDrive\010_NISA'
path0 = path0.replace('\\', '/')

In [5]:
T = 'PFXF'
url = 'https://www.vaneck.com/us/en/investments/preferred-securities-ex-financials-etf-pfxf/downloads/holdings/'
response = requests.get(url, timeout=60)

name = r'tmp.xlsx'
path = path0 + '/' + T + '/'  + name

with open(path,'wb') as f:
    f.write(response.content)
    
df = pd.read_excel(path)
i = df[df.iloc[:,0]=='Number'].index[0]
os.remove(path)

name = df.columns[0]
name = name.split(' ')[-1]
month, day, year = name.split('/')
name = f"{year}{month}{day}"

name = T + '_' + name + '.xlsx'

path = path0 + '/' + T + '/'  + name
with open(path,'wb') as f:
    f.write(response.content)

for i in range(10):
    if df.iloc[i,0] == 'Number':
        break

df = pd.read_excel(path, skiprows=i+1)
print('Ticker:',T)

tmp = sum(df['% of Net Assets'].str.rstrip('%').astype('float') / 100)
print('Net Weithg:',"{:.2f}%".format(tmp * 100))

print('Number of Data:',len(df))
tmp = df['Asset Class']
L = tmp.tolist()
L = {item: L.count(item) for item in set(L)}
L = sorted(set(L), key=lambda x: (-L[x], x))

for i in L:
    tmp1 = len(df.loc[tmp.isin([i])])
    print(' '+str(i)+':',tmp1)

EQ = df.loc[df['Asset Class'].isin(['Stock']),'Identifier (FIGI)'].tolist() 
Pfd = [x + ' US Pfd' for x in EQ]
Pfd = blp.bdp(Pfd,'market_sector_des')
print('\n'+'Breakdown of Stock')
print(' Pfd:',len(Pfd.loc[Pfd['market_sector_des']=='Pfd',:]))

Ticker: PFXF
Net Weithg: 100.01%
Number of Data: 107
 Stock: 105
 Cash: 1
 Cash Bal: 1

Breakdown of Stock
 Pfd: 105


In [6]:
T = 'SRLN'
url = 'https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-srln.xlsx'
response = requests.get(url, timeout=60)

path = r'C:\Users\ky090\OneDrive - The University of Texas at Austin\102_TAM\071_GETF\010_NISA' +'/' + T
path = path.replace('\\', '/')
name = r'tmp.xlsx'
path = path0 + '/' + T + '/'  + name

with open(path,'wb') as f:
    f.write(response.content)
    
df = pd.read_excel(path)
i = df[df.iloc[:,0]=='Name'].index[0]
os.remove(path)


tmp = df.loc[df['Fund Name:']=='Holdings:','SPDR® Blackstone Senior Loan ETF'].values[0]
pattern = r"As of (\d{2})-(\w{3})-(\d{4})"
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}
def replace_with_date(match):
    day, month_abbr, year = match.groups()
    month = month_map.get(month_abbr.capitalize(), '01')
    return f"{year}{month}{day}"
name = re.sub(pattern, replace_with_date, tmp)
name = T + '_' + name + '.xlsx'


path = path0 + '/' + T + '/'  + name
with open(path,'wb') as f:
    f.write(response.content)


df = pd.read_excel(path, skiprows=i+1)
i = df.loc[df['Name']=='NET CASH',:].index[0]
df = df.loc[:i-1,:]

print('Ticker:',T)
print('Net Weithg:',"{:.2f}%".format(sum(df['Weight'])))
print('Number of Data:',len(df))

ID = [x + ' Corp' for x in df['FIGI']]
SB = blp.bdp(ID,['lqa_liquidity_sector','Name'])
No_ID = [i for i in ID if i not in SB.index.tolist()]


print('\n'+'Breakdown of Corp')
net = 0
tmp = SB['lqa_liquidity_sector']
L = tmp.tolist()
L = {item: L.count(item) for item in set(L)}
L = sorted(set(L), key=lambda x: (-L[x], x))

for i in L:
    tmp1 = len(SB.loc[tmp.isin([i])])
    print(' '+str(i)+':',tmp1)

print(' No_ID',len(No_ID))

Ticker: SRLN
Net Weithg: 100.00%
Number of Data: 551

Breakdown of Corp
 CORP HY NA: 267
 INST LOANS: 263
 nan: 10
 CORP IG NA: 5
 No_ID 6
